In [ ]:
import numpy as np
import pandas as pd
import json
from pandas import json_normalize

In [ ]:
with open("/content/data.json", 'r') as file:
    data = json.load(file)


In [ ]:
data

{'metadata': {'api_version': '1.3', 'last_updated': '2023-01-20'},
 'data': {'company': {'cin': 'U72200KA2XXXXXX',
   'legal_name': 'TEST PRIVATE LIMITED',
   'efiling_status': 'Active',
   'incorporation_date': '2014-09-03',
   'paid_up_capital': 1179.76,
   'sum_of_charges': 0,
   'authorized_capital': 2000000,
   'active_compliance': 'ACTIVE compliant',
   'cirp_status': None,
   'lei': {'number': None,
    'status': None,
    'registration_date': None,
    'last_updated_date': None,
    'next_renewal_date': None},
   'registered_address': {'full_address': 'Sector 1 Bengaluru Bangalore KA 560102 IN',
    'address_line1': ' ',
    'address_line2': 'Sector 1',
    'city': 'BENGALURU',
    'pincode': '560102',
    'state': 'KARNATAKA'},
   'business_address': {'address_line1': '27th Main Road,',
    'address_line2': 'Sector 1, ',
    'city': 'Bengaluru',
    'pincode': 560102,
    'state': 'Karnataka'},
   'pan': 'xx',
   'website': None,
   'classification': 'Private Limited Indian No

In [ ]:
d=data['data']

In [ ]:
d.get('financials')

{'year': '2022-03-31',
 'nature': 'STANDALONE',
 'stated_on': '2022-03-31',
 'filing_type': 'E_FORM',
 'filing_standard': 'Schedule III',
 'bs': {'assets': {'tangible_assets': 71440,
   'producing_properties': None,
   'intangible_assets': 1351310,
   'preproducing_properties': None,
   'tangible_assets_capital_work_in_progress': 0,
   'intangible_assets_under_development': 0,
   'noncurrent_investments': 674280,
   'deferred_tax_assets_net': 0,
   'foreign_curr_monetary_item_trans_diff_asset_account': None,
   'long_term_loans_and_advances': 0,
   'other_noncurrent_assets': 0,
   'current_investments': 0,
   'inventories': 0,
   'trade_receivables': 517220,
   'cash_and_bank_balances': 400330,
   'short_term_loans_and_advances': 0,
   'other_current_assets': 5170,
   'given_assets_total': 3019750},
  'liabilities': {'share_capital': 1179760,
   'reserves_and_surplus': -1486960,
   'money_received_against_share_warrants': 0,
   'share_application_money_pending_allotment': 0,
   'deferr

In [ ]:
class FLAGS:
    GREEN = 1
    AMBER = 2
    RED = 0
    MEDIUM_RISK = 3  # diplay purpose only
    WHITE = 4  # data is missing for this field

# This is a already written for your reference

In [ ]:
def latest_financial_index(data: dict):
    """
    Determine the index of the latest standalone financial entry in the data.

    This function iterates over the "financials" list in the given data dictionary.
    It returns the index of the first financial entry where the "nature" key is equal to "STANDALONE".
    If no standalone financial entry is found, it returns 0.

    Parameters:
    - data (dict): A dictionary containing a list of financial entries under the "financials" key.

    Returns:
    - int: The index of the latest standalone financial entry or 0 if not found.
    """
    for index, financial in enumerate(data.get("financials")):
        if financial.get("nature") == "STANDALONE":
            return index
    return 0

In [ ]:
def total_revenue(data: dict, financial_index: int):
    """
    Calculate the total revenue from the financial data at the given index.

    This function accesses the "financials" list in the data dictionary at the specified index.
    It then retrieves the net revenue from the "pnl" (Profit and Loss) section under "lineItems".

    Parameters:
    - data (dict): A dictionary containing financial data.
    - financial_index (int): The index of the financial entry to be used for calculation.

    Returns:
    - float: The net revenue value from the financial data.
    """
    financial_entry = data['data'].get('financials', [])[financial_index]
    pnl_section = financial_entry.get('pnl', {})
    line_items = pnl_section.get('lineItems', {})

    # Assuming 'net_revenue' is the key for net revenue in the lineItems
    net_revenue = line_items.get('net_revenue', 0.0)

    return net_revenue

# Example usage:
# Assuming you want to calculate the total revenue for the financial entry at index 0
financial_index_to_use = 0
total_revenue_value = total_revenue(data, financial_index_to_use)

print(f"Total Revenue: {total_revenue_value}")


Total Revenue: 6717840


In [ ]:
def total_borrowing(data: dict, financial_index: int):
    """
    Calculate the ratio of total borrowings to total revenue for the financial data at the given index.

    This function sums the long-term and short-term borrowings from the balance sheet ("bs")
    section of the financial data. It then divides this sum by the total revenue, calculated
    by calling the `total_revenue` function.

    Parameters:
    - data (dict): A dictionary containing financial data.
    - financial_index (int): The index of the financial entry to be used for calculation.

    Returns:
    - float: The ratio of total borrowings to total revenue.
    """
    # Function to calculate total revenue
    def total_revenue(data: dict, financial_index: int):
        financial_entry = data['data'].get('financials', [])[financial_index]
        pnl_section = financial_entry.get('pnl', {})
        line_items = pnl_section.get('lineItems', {})
        net_revenue = line_items.get('net_revenue', 0.0)
        return net_revenue

    # Accessing balance sheet and liabilities
    balance_sheet = data['data'].get('bs', {})
    liabilities = balance_sheet.get('liabilities', {})

    # Sum of long-term and short-term borrowings
    total_borrowings = liabilities.get('long_term_borrowings', 0) + liabilities.get('short_term_borrowings', 0)

    # Calculate total revenue using the total_revenue function
    total_revenue_value = total_revenue(data, financial_index)

    # Calculate the ratio
    ratio = total_borrowings / total_revenue_value if total_revenue_value != 0 else 0.0

    return ratio

# Assuming you want to calculate the ratio for the financial entry at index 0
financial_index_to_use = 0
borrowing_ratio = total_borrowing(data, financial_index_to_use)

print(f"Ratio of Total Borrowings to Total Revenue: {borrowing_ratio}")


Ratio of Total Borrowings to Total Revenue: 0.0


In [ ]:
class FLAGS:
    GREEN = 1
    AMBER = 2
    RED = 0
    MEDIUM_RISK = 3  # display purpose only
    WHITE = 4  # data is missing for this field

def iscr(data: dict, financial_index: int):
    """
    Calculate the Interest Service Coverage Ratio (ISCR) for the financial data at the given index.

    This function computes the ISCR using the formula: ISCR = Operating Profit / Interest

    Parameters:
    - data (dict): A dictionary containing financial data.
    - financial_index (int): The index of the financial entry to be used for the ISCR calculation.

    Returns:
    - float: The calculated ISCR value.
    """
    financial_entry = data['data'].get('financials', [])[financial_index]
    pnl_section = financial_entry.get('pnl', {})
    line_items = pnl_section.get('lineItems', {})

    # Assuming 'operating_profit' and 'interest' are the keys for operating profit and interest in lineItems
    operating_profit = line_items.get('operating_profit', 0.0)
    interest = line_items.get('interest', 0.0)

    # Calculate ISCR
    iscr_value = operating_profit / interest if interest != 0 else 0.0

    return iscr_value

def iscr_flag(data: dict, financial_index: int):
    """
    Determine the flag color based on the Interest Service Coverage Ratio (ISCR) value.

    This function calculates the ISCR value by calling the `iscr` function and then assigns a flag color
    based on the ISCR value. If the ISCR value is greater than or equal to 2, it assigns a GREEN flag,
    otherwise, it assigns a RED flag.

    Parameters:
    - data (dict): A dictionary containing financial data.
    - financial_index (int): The index of the financial entry to be used for the ISCR calculation.

    Returns:
    - int: The flag color based on the ISCR value (FLAGS.GREEN or FLAGS.RED).
    """
    iscr_value = iscr(data, financial_index)

    if iscr_value >= 2:
        return FLAGS.GREEN
    else:
        return FLAGS.RED

# Assuming you want to calculate the flag color for the financial entry at index 0
financial_index_to_use = 0
flag_color = iscr_flag(data, financial_index_to_use)

print(f"ISCR Flag Color: {flag_color}")


ISCR Flag Color: 0


In [ ]:
class FLAGS:
    GREEN = 1
    RED = 0

def total_revenue_5cr_flag(data: dict, financial_index: int):
    """
    Determine the flag color based on whether the total revenue exceeds 50 million.

    This function calculates the total revenue by calling the `total_revenue` function and then assigns
    a flag color based on the revenue amount. If the total revenue is greater than or equal to 50 million,
    it assigns a GREEN flag, otherwise, it assigns a RED flag.

    Parameters:
    - data (dict): A dictionary containing financial data.
    - financial_index (int): The index of the financial entry to be used for the revenue calculation.

    Returns:
    - FLAGS.GREEN or FLAGS.RED: The flag color based on the total revenue.
    """
    # Assuming that total_revenue function is already defined
    total_revenue_value = total_revenue(data, financial_index)

    # Check if total revenue exceeds 50 million
    if total_revenue_value >= 50000000:
        return FLAGS.GREEN
    else:
        return FLAGS.RED

# Assuming you want to calculate the flag for the financial entry at index 0
financial_index_to_use = 0
revenue_flag = total_revenue_5cr_flag(data, financial_index_to_use)

if revenue_flag == FLAGS.GREEN:
    print("Total Revenue Exceeds 50 Million: GREEN FLAG")
else:
    print("Total Revenue Less Than 50 Million: RED FLAG")


Total Revenue Less Than 50 Million: RED FLAG


In [ ]:
def iscr(data: dict, financial_index: int):
    """
    Calculate the Interest Service Coverage Ratio (ISCR) for the financial data at the given index.

    ISCR is a ratio that measures how well a company can cover its interest payments on outstanding debt.
    It is calculated as the sum of profit before interest and tax, and depreciation, divided by the sum
    of interest expenses. The division is avoided by checking if interest expenses are non-zero.

    Parameters:
    - data (dict): A dictionary containing financial data.
    - financial_index (int): The index of the financial entry to be used for the ISCR calculation.

    Returns:
    - float: The ISCR value.
    """
    financial_entry = data['data'].get('financials', [])[financial_index]
    pnl_section = financial_entry.get('pnl', {})
    line_items = pnl_section.get('lineItems', {})

    # Assuming 'profit_before_interest_and_tax' is the key for profit before interest and tax
    profit_before_interest_and_tax = line_items.get('profit_before_interest_and_tax', 0.0)

    # Assuming 'depreciation' is the key for depreciation
    depreciation = line_items.get('depreciation', 0.0)

    # Assuming 'interest' is the key for interest expenses
    interest_expenses = line_items.get('interest', 0.0)

    # Calculate ISCR only if interest expenses are non-zero to avoid division by zero
    iscr_value = (profit_before_interest_and_tax + depreciation) / interest_expenses if interest_expenses != 0 else 0.0

    return iscr_value

# Assuming you want to calculate ISCR for the financial entry at index 0
financial_index_to_use = 0
iscr_value = iscr(data, financial_index_to_use)

print(f"Interest Service Coverage Ratio (ISCR): {iscr_value}")


Interest Service Coverage Ratio (ISCR): 0.0


In [ ]:
class FLAGS:
    GREEN = 1
    AMBER = 2
    # RED = 0
    MEDIUM_RISK = 3  # display purpose only
    WHITE = 4  # data is missing for this field

def total_borrowing(data: dict, financial_index: int):
    """
    Calculate the ratio of total borrowings to total revenue for the financial data at the given index.

    This function sums the long-term and short-term borrowings from the balance sheet ("bs")
    section of the financial data. It then divides this sum by the total revenue, calculated
    by calling the `total_revenue` function.

    Parameters:
    - data (dict): A dictionary containing financial data.
    - financial_index (int): The index of the financial entry to be used for calculation.

    Returns:
    - float: The ratio of total borrowings to total revenue.
    """
    balance_sheet = data['data'].get('bs', {})
    liabilities = balance_sheet.get('liabilities', {})

    # Sum of long-term and short-term borrowings
    total_borrowings = liabilities.get('long_term_borrowings', 0) + liabilities.get('short_term_borrowings', 0)

    # Calculate total revenue using the total_revenue function
    total_revenue_value = total_revenue(data, financial_index)

    # Calculate the ratio
    ratio = total_borrowings / total_revenue_value if total_revenue_value != 0 else 0.0

    return ratio

def borrowing_to_revenue_flag(data: dict, financial_index: int):
    """
    Determine the flag color based on the ratio of total borrowings to total revenue.

    This function calculates the ratio of total borrowings to total revenue by calling the `total_borrowing`
    function and then assigns a flag color based on the calculated ratio. If the ratio is less than or equal
    to 0.25, it assigns a GREEN flag, otherwise, it assigns an AMBER flag.

    Parameters:
    - data (dict): A dictionary containing financial data.
    - financial_index (int): The index of the financial entry to be used for the ratio calculation.

    Returns:
    - FLAGS.GREEN or FLAGS.AMBER: The flag color based on the borrowing to revenue ratio.
    """
    borrowing_ratio = total_borrowing(data, financial_index)

    if borrowing_ratio <= 0.25:
        return FLAGS.GREEN
    else:
        return FLAGS.AMBER

# Assuming you want to calculate the flag for the financial entry at index 0
financial_index_to_use = 0
flag_color = borrowing_to_revenue_flag(data, financial_index_to_use)

print(f"Flag Color: {flag_color}")


Flag Color: 1


In [ ]:
class FLAGS:
    GREEN = 1
    AMBER = 2
    # RED = 0
    MEDIUM_RISK = 3  # diplay purpose only
    WHITE = 4  # data is missing for this field

def total_revenue(data: dict, financial_index: int):
    """
    Calculate the total revenue from the financial data at the given index.

    This function accesses the "financials" list in the data dictionary at the specified index.
    It then retrieves the net revenue from the "pnl" (Profit and Loss) section under "lineItems".

    Parameters:
    - data (dict): A dictionary containing financial data.
    - financial_index (int): The index of the financial entry to be used for calculation.

    Returns:
    - float: The net revenue value from the financial data.
    """
    financial_entry = data['data'].get('financials', [])[financial_index]
    pnl_section = financial_entry.get('pnl', {})
    line_items = pnl_section.get('lineItems', {})

    # Assuming 'net_revenue' is the key for net revenue in the lineItems
    net_revenue = line_items.get('net_revenue', 0.0)

    return net_revenue

def total_borrowing(data: dict, financial_index: int):
    """
    Calculate the ratio of total borrowings to total revenue for the financial data at the given index.

    This function sums the long-term and short-term borrowings from the balance sheet ("bs")
    section of the financial data. It then divides this sum by the total revenue, calculated
    by calling the `total_revenue` function.

    Parameters:
    - data (dict): A dictionary containing financial data.
    - financial_index (int): The index of the financial entry to be used for calculation.

    Returns:
    - float: The ratio of total borrowings to total revenue.
    """
    balance_sheet = data['data'].get('bs', {})
    liabilities = balance_sheet.get('liabilities', {})

    # Sum of long-term and short-term borrowings
    total_borrowings = liabilities.get('long_term_borrowings', 0) + liabilities.get('short_term_borrowings', 0)

    # Calculate total revenue using the total_revenue function
    total_revenue_value = total_revenue(data, financial_index)

    # Calculate the ratio
    ratio = total_borrowings / total_revenue_value if total_revenue_value != 0 else 0.0

    return ratio

def borrowing_to_revenue_flag(data: dict, financial_index: int):
    """
    Determine the flag color based on the ratio of total borrowings to total revenue.

    This function calculates the ratio of total borrowings to total revenue by calling the `total_borrowing`
    function and then assigns a flag color based on the calculated ratio. If the ratio is less than or equal
    to 0.25, it assigns a GREEN flag, otherwise, it assigns an AMBER flag.

    Parameters:
    - data (dict): A dictionary containing financial data.
    - financial_index (int): The index of the financial entry to be used for the ratio calculation.

    Returns:
    - FLAGS.GREEN or FLAGS.AMBER: The flag color based on the borrowing to revenue ratio.
    """
    ratio = total_borrowing(data, financial_index)

    if ratio <= 0.25:
        return FLAGS.GREEN
    else:
        return FLAGS.AMBER

# Assuming you want to calculate the flag for the financial entry at index 0
financial_index_to_use = 0
flag_color = borrowing_to_revenue_flag(data, financial_index_to_use)

print(f"Flag Color: {flag_color}")


Flag Color: 1
